In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from numpy import random as rand
import pandas as pd
from math import e
from tqdm import tqdm
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D

In [2]:
#G is a graph cointanining N nodes, each of which has two variables: α and s (s = 0 for defection, s = 1 for cooperation)

def networkRGG_cumulative(G,N,macrosteps,pars):
    
    vec_a_mean=[]           #empty vector to be filled with the mean of alpha at each macrostep loop
    vec_x=[]                #empty vector to be filled with the fraction of cooperators at each macrostep loop
    check=[]
    a_vec = list(nx.get_node_attributes(G, 'alpha').values())       #list of alpha of all nodes, different at every macrostep
    s_vec = list(nx.get_node_attributes(G, 'strategy').values())    #list of strategy of all nodes, different at every macrostep
 
    neighbours = [list(nx.all_neighbors(G, node)) for node in range(N)]
    
    for t in tqdm(macrosteps, desc = 'Progress Bar'):
        
        nx.set_node_attributes(G,{ n: a_vec[n] for n in range(N) },'alpha')
        nx.set_node_attributes(G,{ n: s_vec[n] for n in range(N) },'strategy')
        
        vec_a_mean = np.append(vec_a_mean, np.mean(a_vec))       #vector with mean of alpha at each macrostep
        vec_x = np.append(vec_x, np.mean(s_vec))                 #vector with fraction of cooperators at each macrostep

        s_vec=[]
        a_vec=[]
        cumulative_payoffs=[]  
        cum_x=[]
        
        #calculate the cumulative payoff and x_neighbours for all players
        for node in range(N):
            a1,s1 = G.nodes[node]['alpha'] , G.nodes[node]['strategy']
            pm1=pd.DataFrame(np.array([[-pars['p']*pars['cd']*(1-pars['e'])*(1-pars['d'])-(1-a1)*pars['cn'], -pars['p']*pars['cd']*(1-pars['e'])-(1-a1)*pars['cn']],[-pars['p']*pars['cd']*(1-pars['d']), -pars['p']*pars['cd']]]), index = [1, 0], columns=[1, 0])
            pi=0
            sum_x_neigh = 0
            
            for neighbor in neighbours[node]:
                s2 = G.nodes[neighbor]['strategy']
                pi =  pi + pm1.at[int(s1),int(s2)]
                sum_x_neigh = sum_x_neigh + s2
                
            cumulative_payoffs = np.append(cumulative_payoffs,pi)
            if len(neighbours[node])>0:
                cum_x = np.append(cum_x,sum_x_neigh/len(neighbours[node]))
            else: 
                cum_x = np.append(cum_x,s1)
        
        for pos1 in range(N):
            
            if len(neighbours[pos1])==0: 
                a1,s1 = G.nodes[pos1]['alpha'],G.nodes[pos1]['strategy'] 
                s_vec=np.append(s_vec,s1)
                a_vec=np.append(a_vec,a1)
                continue
            
            a1,s1 = G.nodes[pos1]['alpha'] , G.nodes[pos1]['strategy']  
            pos2 = rand.choice(list(neighbours[pos1]))
            s2 = G.nodes[pos2]['strategy']
            
            pi_1=cumulative_payoffs[pos1]
            pi_2=cumulative_payoffs[pos2]
            delta_pi = pi_2-pi_1              

            prob = 1/(1+e**(-beta*delta_pi))
            
            a1_new = a1 + pars['gamma']*(2*(cum_x[pos1])-1)*a1*(1-a1)
            if a1_new>=1: a1_new=0.99
            if a1_new<=0: a1_new=0.01

            if rand.random()<prob: s1_new = s2
            else: s1_new = s1
            
            a_vec=np.append(a_vec,a1_new)
            s_vec=np.append(s_vec,s1_new)
     
    return G, vec_x, vec_a_mean, np.mean(s_vec),np.mean(a_vec)

In [ ]:
#Parameters
pars={} 
pars['cd']=1
pars['e']=0.9
pars['d']=0.5
pars['gamma']=1

beta=1
N=300
n_coop=150
radius=0.083
nums = np.zeros(N) 
nums[:n_coop] = 1
xm=[[0]*41 for _ in range(41)]
am=[[0]*41 for _ in range(41)]
i=0
for pars['cn'] in np.linspace(0,1,41):    
    j=0
    
    for pars['p'] in np.linspace(0,1,41):
        rand.seed()
        np.random.shuffle(nums)
        vector=np.linspace(0.01,0.99,N)
        np.random.shuffle(vector)

        G = nx.random_geometric_graph(N, radius, dim=2, pos=None, p=2, seed=None)
        nx.set_node_attributes(G,{ n: vector[n] for n in range(N) },'alpha')
        nx.set_node_attributes(G,{ n: nums[n] for n in range(N)  },'strategy')

        macrosteps = list(range(300))

        r= networkRGG_cumulative(G,N,macrosteps,pars)
        
        xm[i][j]=r[3]
        am[i][j]=r[4]
        j=j+1
        
    i=i+1
    
pd.DataFrame(xm).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/x_rgg_300_41_5.tsv", sep=" ",index=False,header=False)
pd.DataFrame(am).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/a_rgg_300_41_5.tsv", sep=" ",index=False,header=False)

Progress Bar:  44%|███████████▉               | 133/300 [00:31<00:39,  4.28it/s]

In [ ]:
#Parameters
pars={} 
pars['cd']=1
pars['e']=0.9
pars['d']=0.5
pars['gamma']=1

beta=1
N=300
n_coop=150
radius=0.083
nums = np.zeros(N) 
nums[:n_coop] = 1
xm=[[0]*41 for _ in range(41)]
am=[[0]*41 for _ in range(41)]
i=0
for pars['cn'] in np.linspace(0,1,41):    
    j=0
    
    for pars['p'] in np.linspace(0,1,41):
        rand.seed()
        np.random.shuffle(nums)
        vector=np.linspace(0.01,0.99,N)
        np.random.shuffle(vector)

        G = nx.random_geometric_graph(N, radius, dim=2, pos=None, p=2, seed=None)
        nx.set_node_attributes(G,{ n: vector[n] for n in range(N) },'alpha')
        nx.set_node_attributes(G,{ n: nums[n] for n in range(N)  },'strategy')

        macrosteps = list(range(300))

        r= networkRGG_cumulative(G,N,macrosteps,pars)
        
        xm[i][j]=r[3]
        am[i][j]=r[4]
        j=j+1
        
    i=i+1
    
pd.DataFrame(xm).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/x_rgg_300_41_6.tsv", sep=" ",index=False,header=False)
pd.DataFrame(am).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/a_rgg_300_41_6.tsv", sep=" ",index=False,header=False)

In [ ]:
#Parameters
pars={} 
pars['cd']=1
pars['e']=0.9
pars['d']=0.5
pars['gamma']=1

beta=1
N=300
n_coop=150
radius=0.083
nums = np.zeros(N) 
nums[:n_coop] = 1
xm=[[0]*41 for _ in range(41)]
am=[[0]*41 for _ in range(41)]
i=0
for pars['cn'] in np.linspace(0,1,41):    
    j=0
    
    for pars['p'] in np.linspace(0,1,41):
        rand.seed()
        np.random.shuffle(nums)
        vector=np.linspace(0.01,0.99,N)
        np.random.shuffle(vector)

        G = nx.random_geometric_graph(N, radius, dim=2, pos=None, p=2, seed=None)
        nx.set_node_attributes(G,{ n: vector[n] for n in range(N) },'alpha')
        nx.set_node_attributes(G,{ n: nums[n] for n in range(N)  },'strategy')

        macrosteps = list(range(300))

        r= networkRGG_cumulative(G,N,macrosteps,pars)
        
        xm[i][j]=r[3]
        am[i][j]=r[4]
        j=j+1
        
    i=i+1
    
pd.DataFrame(xm).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/x_rgg_300_41_7.tsv", sep=" ",index=False,header=False)
pd.DataFrame(am).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/a_rgg_300_41_7.tsv", sep=" ",index=False,header=False)

In [ ]:
#Parameters
pars={} 
pars['cd']=1
pars['e']=0.9
pars['d']=0.5
pars['gamma']=1

beta=1
N=300
n_coop=150
radius=0.083
nums = np.zeros(N) 
nums[:n_coop] = 1
xm=[[0]*41 for _ in range(41)]
am=[[0]*41 for _ in range(41)]
i=0
for pars['cn'] in np.linspace(0,1,41):    
    j=0
    
    for pars['p'] in np.linspace(0,1,41):
        rand.seed()
        np.random.shuffle(nums)
        vector=np.linspace(0.01,0.99,N)
        np.random.shuffle(vector)

        G = nx.random_geometric_graph(N, radius, dim=2, pos=None, p=2, seed=None)
        nx.set_node_attributes(G,{ n: vector[n] for n in range(N) },'alpha')
        nx.set_node_attributes(G,{ n: nums[n] for n in range(N)  },'strategy')

        macrosteps = list(range(300))

        r= networkRGG_cumulative(G,N,macrosteps,pars)
        
        xm[i][j]=r[3]
        am[i][j]=r[4]
        j=j+1
        
    i=i+1
    
pd.DataFrame(xm).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/x_rgg_300_41_8.tsv", sep=" ",index=False,header=False)
pd.DataFrame(am).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/a_rgg_300_41_8.tsv", sep=" ",index=False,header=False)

In [ ]:
#Parameters
pars={} 
pars['cd']=1
pars['e']=0.9
pars['d']=0.5
pars['gamma']=1

beta=1
N=300
n_coop=150
radius=0.083
nums = np.zeros(N) 
nums[:n_coop] = 1
xm=[[0]*41 for _ in range(41)]
am=[[0]*41 for _ in range(41)]
i=0
for pars['cn'] in np.linspace(0,1,41):    
    j=0
    
    for pars['p'] in np.linspace(0,1,41):
        rand.seed()
        np.random.shuffle(nums)
        vector=np.linspace(0.01,0.99,N)
        np.random.shuffle(vector)

        G = nx.random_geometric_graph(N, radius, dim=2, pos=None, p=2, seed=None)
        nx.set_node_attributes(G,{ n: vector[n] for n in range(N) },'alpha')
        nx.set_node_attributes(G,{ n: nums[n] for n in range(N)  },'strategy')

        macrosteps = list(range(300))

        r= networkRGG_cumulative(G,N,macrosteps,pars)
        
        xm[i][j]=r[3]
        am[i][j]=r[4]
        j=j+1
        
    i=i+1
    
pd.DataFrame(xm).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/x_rgg_300_41_9.tsv", sep=" ",index=False,header=False)
pd.DataFrame(am).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/a_rgg_300_41_9.tsv", sep=" ",index=False,header=False)